# Imports

In [1]:
import dlt
import requests
import pandas as pd
from datetime import datetime

# Q1

In [2]:
# What's the version of dlt that you installed?

In [3]:
!dlt --version

dlt 1.14.1


# Q2

In [4]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [5]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [ ]:
from dlt.destinations.adapters import qdrant_adapter

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"
)

# load_info = pipeline.run(zoomcamp_data())
# print(pipeline.last_trace)

load_info = pipeline.run(
    qdrant_adapter(zoomcamp_data(), embed=["text", "section", "question"])
)
print(pipeline.last_trace)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = pipeline.dataset().zoomcamp_data.df()

In [ ]:
dataset.shape

# Q3